# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
from azureml.core import Workspace, Experiment
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.data.dataset_factory import TabularDatasetFactory
from train import split_data
from sklearn.model_selection import train_test_split
from azureml.core import ScriptRunConfig 
import os

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-project-automl'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

web_path = "https://raw.githubusercontent.com/JoanneJons/azure-machine-learning-capstone/main/breast-cancer-dataset.csv?token=AJ5V2OGXYLJ22BGYXN4EUODAC6P4K"
dataset = TabularDatasetFactory.from_delimited_files(path=web_path)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RTAEEBNB3 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-136084
Azure region: southcentralus
Subscription id: 9b72f9e6-56c5-4c16-991b-19c652994860
Resource group: aml-quickstarts-136084


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "cpucluster"
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [7]:
import pandas as pd
x, y = split_data(dataset)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

try:
    os.makedirs('./data', exist_ok=True)
except OSError as error:
    print('New directory cannot be created')

train_df = X_train
train_df['diagnosis'] = y_train

train_path = 'data/train-data.csv'
train_df.to_csv(train_path)

test_df = X_test
test_df['diagnosis'] = y_test

test_path = 'data/test-data.csv'
test_df.to_csv(test_path)

datastore = ws.get_default_datastore()
datastore.upload(src_dir='data', target_path='data')

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Uploading an estimated of 2 files
Uploading data/test-data.csv
Uploaded data/test-data.csv, 1 files out of an estimated total of 2
Uploading data/train-data.csv
Uploaded data/train-data.csv, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_29f4bddbc56b42ea86d0054a08febd25

In [8]:
train_data = TabularDatasetFactory.from_delimited_files(path=[(datastore, ('data/train-data.csv'))])
test_data = TabularDatasetFactory.from_delimited_files(path=[(datastore, ('data/test-data.csv'))])

In [10]:
train_data

{
  "source": [
    "('workspaceblobstore', 'data/train-data.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [11]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here
# automl_settings = {}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name='diagnosis',
    n_cross_validations=5,
    compute_target=cpu_cluster
)

In [12]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

Running on remote.
No run_configuration provided, running on cpucluster with default configuration
Running on remote compute: cpucluster
Parent Run ID: AutoML_7ce25c35-2267-43dc-b6c4-ffa20042674a

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: ht

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_7ce25c35-2267-43dc-b6c4-ffa20042674a',
 'target': 'cpucluster',
 'status': 'Completed',
 'startTimeUtc': '2021-01-26T07:38:00.954055Z',
 'endTimeUtc': '2021-01-26T08:19:58.551299Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'cpucluster',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone-project-automl","subscription_id":"9b72f9e6-56c5-4c16-991b-19c652994860","resource_group":"aml-quickstarts-136084","workspace_name":"quick-starts-ws-136084","region":"southcentralus","compute_target":"cpucluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":



OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?


## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
best_run, fitted_model = remote_run.get_output()

print(best_run)

Run(Experiment: capstone-project-automl,
Id: AutoML_7ce25c35-2267-43dc-b6c4-ffa20042674a_23,
Type: azureml.scriptrun,
Status: Completed)


In [14]:
print(fitted_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                num_leaves=227,
                                                                                                objective=None,
                                                                                                random_state=None,
                                                     

In [15]:
best_run.get_tags()

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":0,"CurrentNodeCount":1}',
 'ensembled_iterations': '[19, 14, 5, 0, 21]',
 'ensembled_algorithms': "['LogisticRegression', 'GradientBoosting', 'SVM', 'LightGBM', 'LightGBM']",
 'ensemble_weights': '[0.3333333333333333, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666]',
 'best_individual_pipeline_score': '0.9726822767942128',
 'best_individual_iteration': '19',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True',
 'model_explain_run_id': 'AutoML_7ce25c35-2267-43dc-b6c4-ffa20042674a_ModelExplain',
 'model_explanation': 'True'}

In [16]:
metrics = best_run.get_metrics()
metrics['accuracy']

0.9746240243670284

In [24]:
#TODO: Save the best model
import joblib
from azureml.core.model import Model

description = "AutoML model trained on Wisoconsin Breast Cancer dataset"

os.makedirs('outputs', exist_ok=True)
joblib.dump(fitted_model, filename="outputs/automl-model.pkl")
automl_model = remote_run.register_model(model_name='automl-breast-cancer', description=description)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [38]:
from azureml.core.webservice import AciWebservice

aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    description='Predict tumor as Malignant(1) or Benign(0)',
    auth_enabled=True
)

In [39]:
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model

model = Model(ws, 'automl-breast-cancer')


myenv = best_run.get_environment()
entry_script = 'score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', entry_script)

inference_config = InferenceConfig(entry_script=entry_script, environment=myenv)

service = Model.deploy(workspace=ws, 
                       name='automl-breast-cancer', 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aci_config)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.............................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [40]:
service.update(enable_app_insights=True)

In [43]:
print("State "+ service.state)
print("Key " + service.get_keys()[0])
print("Swagger URI " + service.swagger_uri)
print("Scoring URI " + service.scoring_uri)

State Healthy
Key XqAtjrHlZQ6gqknuuRSvN86dRt7QQbzG
Swagger URI http://19ac972d-b1dd-49dc-af6a-db72674ba244.southcentralus.azurecontainer.io/swagger.json
Scoring URI http://19ac972d-b1dd-49dc-af6a-db72674ba244.southcentralus.azurecontainer.io/score


TODO: In the cell below, send a request to the web service you deployed to test it.

In [44]:
# Model endpoint is consumed using a script which contains 2 datapoints
# Expected result [0, 1]
%run endpoint.py

{"result": [0, 1]}


TODO: In the cell below, print the logs of the web service and delete the service

In [45]:
print(service.get_logs())


2021-01-26T10:05:26,275219200+00:00 - gunicorn/run 
2021-01-26T10:05:26,276765100+00:00 - iot-server/run 
2021-01-26T10:05:26,292472100+00:00 - nginx/run 
2021-01-26T10:05:26,283410100+00:00 - rsyslog/run 
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [47]:
service.delete()
cpu_cluster.delete()

No service with name automl-breast-cancer found to delete.
Current provisioning state of AmlCompute is "Deleting"

